In [1]:
%pwd

'd:\\NASA-Battery-Life-Prediction\\notebook'

In [2]:
import os
os.chdir('../')
%pwd

'd:\\NASA-Battery-Life-Prediction'

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    epochs: int
    batch_size: int
    optimizer: str
    loss: str
    validation_split: float
    gru_units: int
    dropout_rate: float
    target_column: str

In [21]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.model_trainer
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            epochs=params.epochs,
            batch_size=params.batch_size,
            optimizer=params.optimizer,
            loss=params.loss,
            validation_split=params.validation_split,
            gru_units=params.gru_units,
            dropout_rate=params.dropout_rate,
            target_column=schema.name
        )
        return model_trainer_config

In [32]:
import os
import time
import pandas as pd
import numpy as np
import joblib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input

In [37]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Prepare data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1).values
        X_test = test_data.drop([self.config.target_column], axis=1).values
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]
        
        seq_length = 2
        num_features = X_train.shape[1]
        # Ensure the total features are divisible by seq_length
        if num_features % seq_length != 0:
            raise ValueError(f"Number of features ({num_features}) must be divisible by sequence length ({seq_length}).")
        
        reshaped_features = num_features // seq_length
        X_train = X_train.reshape(X_train.shape[0], seq_length, reshaped_features)
        X_test = X_test.reshape(X_test.shape[0], seq_length, reshaped_features)
        
        model_GRU = Sequential()
        model_GRU.add(Input(shape=(seq_length, reshaped_features)))  # Use Input layer for the first layer
        model_GRU.add(GRU(units=20, return_sequences=True))
        model_GRU.add(GRU(units=50, activation='relu'))
        model_GRU.add(Dense(units=1, activation=None))


        # Compile model
        model_GRU.compile(
            optimizer=self.config.optimizer,
            loss=self.config.loss,
            metrics=['mae']
        )
        
        history = model_GRU.fit(
            X_train, 
            y_train,
            epochs=self.config.epochs,
            batch_size=self.config.batch_size,
            validation_data=(X_test, y_test),
            verbose=1
        )

        joblib.dump(model_GRU, os.path.join(self.config.root_dir, self.config.model_name))

In [38]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-12-27 04:59:17,081: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-27 04:59:17,086: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-27 04:59:17,091: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-27 04:59:17,094: INFO: common: created directory at: artifacts]
[2024-12-27 04:59:17,096: INFO: common: created directory at: artifacts/model_trainer]
Epoch 1/20
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0279 - mae: 0.0975 - val_loss: 0.0072 - val_mae: 0.0469
Epoch 2/20
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0075 - mae: 0.0507 - val_loss: 0.0096 - val_mae: 0.0745
Epoch 3/20
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0070 - mae: 0.0462 - val_loss: 0.0067 - val_mae: 0.0400
Epoch 4/20
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0074 - mae: 0.0476 - val_loss: 0.0069 - val_mae: 0.0395
Epoch 5/20
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0071 - mae: 0.0454 - val_loss: 0.0